In [1]:
import sys
sys.path.append('./Files')
import os
import matplotlib.pyplot as plt
import time
import macroC as mC 
import numpy as np
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import QuadraticHamiltonian
from qiskit.quantum_info import Operator
JW = JordanWignerMapper()

In [3]:
# Definimos si va a ser archivo de minimización de H o de free energy, el sistema a estudiar y si es con o sin shots.
name='AA2MM2Xfixed' # 'Sulfanol'/'AB'/'AX'/'ABX'/'AMX'/'ABC'/'AMPX'/'AA2MM2X'
shots=None 
method='SLSQP' # 'SLSQP'/'COBYLA'
layers=1
func='FE' # 'H'/'FE'
typetrunc='NoTrunc' # 'NoTrunc','Trunc1','Trunc2','Trunc3','Trunc4'

In [3]:
file ='./Data/'+func+'/'+typetrunc+'/'+name+'-'+method+'-'+'Layers='+str(layers)+'-'+'Shots='+str(shots)+'.dat'

In [4]:
# Creamos la lista vacía de los datos que vamos a leer
data=[i.strip().split() for i in open(file).readlines()]
nbetas=len(data)
betas=[float(data[i][0]) for i in range(nbetas)]
sortindex=np.argsort(betas)
betas=np.sort(betas)
overlaps=[float(data[sortindex[i]][1]) for i in range(nbetas)]
niter=[float(data[sortindex[i]][2]) for i in range(nbetas)]
times=[float(data[sortindex[i]][3]) for i in range(nbetas)]

In [5]:
print(betas)

[1.00000e-03 4.26250e-02 8.42500e-02 1.25875e-01 1.67500e-01 2.09125e-01
 2.50750e-01 2.92375e-01 3.34000e-01 3.75625e-01 4.17250e-01 4.58875e-01
 5.00500e-01 5.42125e-01 5.83750e-01 6.25375e-01 6.67000e-01 7.08625e-01
 7.50250e-01 7.91875e-01 8.33500e-01 8.75125e-01 9.16750e-01 9.58375e-01
 1.00000e+00 1.00000e+00 1.37500e+00 1.75000e+00 2.12500e+00 2.50000e+00
 2.87500e+00 3.25000e+00 3.62500e+00 4.00000e+00 4.37500e+00 4.75000e+00
 5.12500e+00 5.50000e+00 5.87500e+00 6.25000e+00 6.62500e+00 7.00000e+00
 7.37500e+00 7.75000e+00 8.12500e+00 8.50000e+00 8.87500e+00 9.25000e+00
 9.62500e+00 1.00000e+01]


In [6]:
# Elegimos el beta que queremos la representación.
indexbeta=5
beta=betas[indexbeta]
print(beta)

0.209125


In [2]:
#[H,shifts, J, B, offset]=mC.system(name)
[H,shifts, J, B, offset]=mC.aniline()

In [4]:
N_sites=H.num_qubits
# Transformamos la descomposición obtenida a operadores fermiónicos mediante una Jordan-Wigner inversa.
start=time.time()
fermionic_op= mC.reverse_map(H)
end=time.time()
print(end-start)
# Creamos la matriz que guardará los coeficientes de la parte cuadrática del operador fermiónico. 
start=time.time()
num_spin_orbitals = fermionic_op.num_spin_orbitals
h1 = np.zeros((num_spin_orbitals, num_spin_orbitals), dtype=np.complex128)

# Extraemos términos cuadráticos.
for term, coeff in fermionic_op.items():
    if term.count('+') == 1 and term.count('-') == 1:  # Verifica que tenga un operador de creación y uno de destrucción
        # Extraer los índices y los signos de los operadores
        indices = [int(t[2]) for t in term.split()]
        signs = [t[0] for t in term.split()]
        
        # Determinar las posiciones en la matriz según los signos
        if signs == ['+', '-']:  # Caso +_i -_j
            h1[indices[0], indices[1]] += coeff
        elif signs == ['-', '+']:  # Caso -_i +_j
            h1[indices[1], indices[0]] -= coeff  # Cambiar el signo del coeficiente
end=time.time()
print(end-start)
# Obtenemos la matriz de cambio de base a una en la que la parte cuadrática sea diagonal mediante la transformación de Bogoliubov.
# También obtenemos los valores diagonales, las energías orbitales.
start=time.time()
hamiltonian = QuadraticHamiltonian(
    hermitian_part=h1,
    constant=0)
(transformation_matrix,
    orbital_energies,
    transformed_constant,
) = hamiltonian.diagonalizing_bogoliubov_transform()
end=time.time()
print(end-start)
# Definimos la matriz de transformación
start=time.time()
T =transformation_matrix
T_conj=T
# Transformamos el FermionicOp a la nueva base.
fermionic_op_new = mC.transform_fermionic_op(fermionic_op, T, T_conj)
end=time.time()
print(end-start)
# Aplicamos una transformación de Jordan-Wigner sobre el operador fermiónico transformado para devolverlo a su expresión en operadores de espín.
start=time.time()
qubit_op=JW.map(fermionic_op_new.simplify()).chop(1e-6)
end=time.time()
print(end-start)
start=time.time()
ansatz=mC.ansatz_U(qubit_op,N_sites,reps=layers)
end=time.time()
print(end-start)
orbital_energies_fixed=np.array([np.real(fermionic_op_new['+_'+str(i)+' '+'-_'+str(i)]) for i in range(N_sites)])

0.09765982627868652
0.0004825592041015625
0.017655134201049805
5880.172115325928
0.10474634170532227
0.06944775581359863


In [ ]:
energdata=[]
n=int(2**N_sites)
for k in range(nbetas):
    energk=[float(data[sortindex[k]][i+4]) for i in range(n)]
    energdata.append(energk)
energ=np.array(energdata[indexbeta])
lentheta=int(float(data[k][n+4]))
thetadata=[]
for k in range(nbetas):
    thetak=[float(data[sortindex[k]][n+5+i]) for i in range(lentheta)]
    thetadata.append(thetak)
theta=np.array(thetadata[indexbeta])

SW=10
d=10**4
factor=1000
# Calculamos la matriz asociada al circuito unitario para los parámetros obtenidos de la minimización.
U_bound = ansatz.assign_parameters(theta)
U_matrix = Operator(U_bound).data
spcOut, fs, spcOutteo, fsteo=mC.getSpectrum(shifts,J,d,SW,B,offset,energ*factor,U_matrix,T)

In [ ]:
np.sort(energ*factor)

In [ ]:
eigvalues,eigvecs=np.linalg.eig(qubit_op)
sortindexth=np.argsort(eigvalues)
eigvalues[sortindexth]

In [ ]:
# Representamos ambos espectros para comparar.
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo)

In [ ]:
# Representamos ambos espectros para comparar.
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo,xlim=[0.9,1.2])

In [ ]:
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo,xlim=[0.95,1.05])

In [ ]:
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo,xlim=[1.2,1.4])

In [ ]:
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo,xlim=[1.9,2.1])

In [ ]:
mC.reprSpectrum(spcOut, fs, spcOutteo, fsteo,xlim=[2.425,2.575])